In [49]:
stock_symbols = ['ASIANPAINT.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'HCLTECH.NS',
                 'INFY.NS', 'TCS.NS', 'TECHM.NS']
weights = [0.1966473, 0.13984502, 0.21313215, 0.06534316, 0.10815051, 0.18365964, 0.09322223]
stock_dict = dict(zip(stock_symbols, weights))

In [50]:
import pandas as pd
real_prices = pd.read_csv("backtesting_data.csv")

In [51]:
real_prices.rename(columns={'_time': 'Date'}, inplace=True)
real_prices['Date'] = pd.to_datetime(real_prices['Date']).dt.date
real_prices.set_index('Date', inplace=True)

In [52]:
real_prices = real_prices.apply(pd.to_numeric)
real_returns = real_prices.pct_change() * 100
real_returns = real_returns.dropna()

In [53]:
if set(real_returns.columns) != set(stock_dict.keys()):
    raise ValueError("Column names in the DataFrame do not match the keys in the weights dictionary.")
weights_series = pd.Series(stock_dict)
real_returns['Combined Return'] = real_returns.dot(weights_series)
combined_returns_df = real_returns.reset_index()[['Date', 'Combined Return']]
combined_returns_df['Date'] = pd.to_datetime(combined_returns_df['Date']).dt.date
combined_returns_df.set_index('Date', inplace=True)

In [54]:
def annualize_rets(r, periods_per_year):
    compounded_growth = (1+r).prod()
    n_periods = r.shape[0]
    return compounded_growth**(periods_per_year/n_periods)-1

def annualize_vol(r, periods_per_year):
    return r.std()*(periods_per_year**0.5)

In [57]:
combined_returns_df['Cumulative Return'] = (1 + combined_returns_df['Combined Return']).cumprod()

In [58]:
combined_returns_df

,Combined Return,Cumulative Return
Date,,
2023-01-03,0.571411,1.571411e+00
2023-01-04,-0.617912,6.004168e-01
2023-01-05,-1.533414,-3.202709e-01
2023-01-06,-1.560043,1.793654e-01
2023-01-09,1.384201,4.276432e-01
...,...,...
2023-12-22,1.041980,-2.004116e-10
2023-12-26,-0.176205,-1.650980e-10
2023-12-27,1.149367,-3.548563e-10
